In [300]:
## all imports
from IPython.display import HTML
import chardet
import io
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
import os
import urllib.request
from urllib.request import urlopen
from bs4 import BeautifulSoup
from time import sleep
import requests
import operator
import socket
import _pickle
import math
import re # regular expressions

from pandas import Series
import pandas as pd
from pandas import DataFrame

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (20, 10)
import json

import seaborn as sns
sns.set_context("talk")
sns.set_style("white")

from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time
from datetime import datetime

#from secret import *

### FUNCTION TO CREATE WIKIPEDIA YEAR URL

In [301]:
def find_url(row):
    Base_Link_01= "https://en.wikipedia.org/wiki/"
    Base_Link_02= "_AFL_season"
    Variable= row
    link = Base_Link_01 + Variable + Base_Link_02
    return (link)

### FUNCTION: ENSURE ALL MONTH NAMES ARE CONSISTENT (REMOVE SHORTENING)

In [302]:
def correct_month (x):
    first_space_index = x.index(" ")
    day = x[:first_space_index]
    split_month = x.split(" ")[1]
    #print (split_month)
    
    months_tocheck = ["January", "Febuary ", "March", "April", "May", "June", "July", "August", "September", "October", "Novemeber", "December"]
    
    #if any (split_month for month in months_tocheck):
    if split_month in months_tocheck:
        #print ("it is a correct month")
        return (x)
    
       
    else:
        month_dict = {
        "Mar" : "March",
        "Apr" : "April",
        "May" : "May",
        "June" : "June",
        "July" : "July",
        "Aug" : "August",
        "Sep" : "September",
        "Oct" : "October",
        }
        #print (month_dict[split_month])
        #print ("is not a correct minth")
        time = x[first_space_index:].strip()
        try:
            space_index = time.index(" ")
            hours = time[space_index:].strip()
            return (day + " " + month_dict[split_month] + " " + hours)

        except:
            return (day + " " + month_dict[split_month])


### FUNCTION: RETURN GAMES DATE

In [303]:
def the_date (x):
    first_space_index = x.index(" ")
    day = x[:first_space_index]
    return (day)

### FUNCTION: RETURN GAMES MONTH

In [304]:
def the_month (x):
    first_space_index = x.index(" ")
    day = x[:first_space_index]
    split_month = x.split(" ")[1]
    return (split_month)


### FUNCTION: RETURN TIME IN HOURS- PERHAPS CONVERT TO 24 HOUR TIME?

In [305]:
def the_time (x):
    first_space_index = x.index(" ")
    day = x[:first_space_index]
    #print (day)
    time = x[first_space_index:].strip()
    space_index = time.index(" ")
    hours = time[space_index:].strip()
    split_month = x.split(" ")[1]
    return (hours[1:-1])
    #print (split_month)

### FUNCTION: RETURN MATCH DAY IF IT EXISTS

In [306]:
def if_dayexists (x):
    ## we know that if it has a day it will have a comma
    #x = "Thursday, 14 June (5:40 pm)"
    count = x.count(",")
    if count >0:
        date = x.split(",")[0].strip()
        return (date)
    else: 
        date = x
        return ("No Day Data Provided")
    


### FUNCTION: TO REMOVE THE DAY FROM THE DATE

In [307]:
def day_conversion (x):
    ## we know if it has a day it will have a comma
    #x = "Thursday, 14 June (5:40 pm)"
    count = x.count(",")
    if count >0:
        date = x.split(",")[1].strip()
        #print (date)
        #first_colon_index = x.index(",")
        #print (first_colon_index)
        #date = x[first_colon_index+1:].strip()
        return (date)
    else: 
        date = x
        return (date)
    
    

### FUNCTION: TO REMOVE ANY WORDS BEFORE THE DATE (EG. PF1:)

In [308]:
def date_conversion (x):
    #x = "PF1: Friday, 25 September (6:20 pm)"
    #x = "Thursday, 14 June (5:40 pm)"
    count = x.count(":")
    if count >1:
        first_colon_index = x.index(":")
        date = x[first_colon_index+1:].strip()
        return (date)
    else: 
        date = x
        return (date)

### FUNCTION: TO CREATE PANDAS STRIPTIME + DATETIME

In [309]:
def structured_times(x):
    if ':' in x:
        struct_time = time.strptime(x, "%Y %d %B (%I:%M %p)")
        dt = datetime.fromtimestamp(time.mktime(struct_time))
        return (dt)
    else: 
        struct_time = time.strptime(x, "%Y %d %B")
        dt = datetime.fromtimestamp(time.mktime(struct_time))
        return (dt)

### FUNCTION: MATCH SCORES DICTIONARY (SINGLE LINE DATA)

In [310]:
def scores_dictionary (test):
    
    short_home_score = (''.join(text for text in test[1].find_all(text=True) if text.parent.name != "a")).strip()
    home_goals= short_home_score.split('.')[0]
    home_behinds= short_home_score.split('.')[1].split('(')[0]
    home_total = (short_home_score.split('.')[1].split('(')[1])[:-1]
    short_away_score = (''.join(text for text in test[3].find_all(text=True) if text.parent.name != "a")).strip()
    away_goals= short_away_score.split('.')[0]
    away_behinds= short_away_score.split('.')[1].split('(')[0]
    away_total = (short_away_score.split('.')[1].split('(')[1])[:-1]
    
    
    ind_match_dict= {
         "Home Goals": home_goals,
         "Home Behinds": home_behinds,
         "Home Total": home_total,
         "Away Goals": away_goals,
         "Away Behinds": away_behinds,
         "Away Total": away_total}

    return (ind_match_dict)

### FUNCTION: MATCH SCORES DICTIONARY (MULTI LINE DATA- GRAND FINALS)

In [324]:
def scores_dictionary_02 (match_goals):
    

    test_data = match_goals.find_all('td')
    home_score = test_data[1].find('span').string
    home_total =  (home_score.split(" ")[1][1:-1])
    home_goals = (home_score.split(" ")[0]).split(".")[0]
    home_behinds = (home_score.split(" ")[0]).split(".")[1]
    away_score = test_data[3].find('span').string
    away_total =  (away_score.split(" ")[1][1:-1])
    away_goals = (away_score.split(" ")[0]).split(".")[0]
    away_behinds = (away_score.split(" ")[0]).split(".")[1]

    
    ind_match_dict= {
         "Home Goals": home_goals,
         "Home Behinds": home_behinds,
         "Home Total": home_total,
         "Away Goals": away_goals,
         "Away Behinds": away_behinds,
         "Away Total": away_total}

    return (ind_match_dict)

### FUNCTION: TYPE A- GENERAL MATCH INFO  DICTIONARY

In [321]:
def match_dictionary_A(test,this_round, this_year):
    #print (test)
    # FUNCTION TAKE OFF FRONT PREFIX
    time_date= date_conversion(test[0].string)


    ### FUNCTION TO ASSIGN DAY IF IT EXISTS (BEFORE WE STRIP IT OFF FOR CLARITY AS NOT NEEDED)
    day = if_dayexists (time_date)


    ### FUNCTION TO TAKE OFF THE DAY AT THE START (AS NOT CONSISTENT)
    convert_time = day_conversion(time_date)
    #print (convert_time)

    ### FUNCTION TO ENSURE CONSISTENT MONTH SYNTAX
    correct_time_syn = correct_month (convert_time)



    
    
    ### FUNCTION TO ASSIGN DATE, MONTH AND TIME
    date = the_date(correct_time_syn)
    month = the_month(correct_time_syn)
    if ':' in correct_time_syn:
        time = the_time (correct_time_syn)
    else: 
        time = "No time provided"


    ### ADDS THE CURRENT YEAR
    convert_time_year = (str(this_year)+ " " + convert_time)
    


    ### FUNCTION THAT CONVERTS TO TIME STRUCTURE
    struct_time = structured_times(convert_time_year)

    home_team = test[1].find('a').string
    away_team = test[3].find('a').string
    venue = test[4].find('a').string
    venue_crowd = (''.join(text for text in test[4].find_all(text=True) if text.parent.name != "a")).strip()
    venue_crowd_= (venue_crowd.split(':')[1])[:-1].strip()
    final_crowd = venue_crowd_.replace(",", "")
    
    try: 
        match_report_url = test[5].find('a')['href']
        
    except:
        match_report_url = "no match report"
   

    match_dict= {
     "Day": day,
     "Date": date,
     "Month": month,
     "Year": this_year,
     "Round": this_round,         
     "Time": time,
     "Structured Data": struct_time,
     "Home Team": home_team,
     "Away Team": away_team,
     "Venue": venue,
     "Crowd": final_crowd,
     "Match Link": match_report_url}
    
    return (match_dict)

### FUNCTION: TYPE B- GENERAL MATCH INFO  DICTIONARY

In [322]:
def match_dictionary_B(test, this_round, this_year, match_report):


    time_date= date_conversion(test[6].string)

    day = if_dayexists (time_date)

    convert_time = day_conversion(time_date)

    convert_time_year = (str(this_year)+ " " + convert_time)
    
    
    date = convert_time.split(" ")[0]
    month = convert_time.split(" ")[1]
    struct_time = structured_times(convert_time_year)
    home_team = test[0].find('a').string
    
    try:
        home_score = test[1].find('b').string
        
    except:
        ## get rid of a child tag within to get out the string
        home_score = (''.join(text for text in test[1].find_all(text=True,recursive=False) if text.parent.name != "small")).strip()

     
        
    home_goals = (home_score.split(" ")[0]).split(".")[0]
    home_behinds = (home_score.split(" ")[0]).split(".")[1]
    home_total = (home_score.split(" ")[1][1:-1])  
    

    away_team = test[2].find('a').string
    
    try:
        away_score = test[3].find('b').string
        
    except:
        ## get rid of a child tag within to get out the string
        away_score = (''.join(text for text in test[3].find_all(text=True,recursive=False) if text.parent.name != "small")).strip()
        

    away_goals = (away_score.split(" ")[0]).split(".")[0]
    away_behinds = (away_score.split(" ")[0]).split(".")[1]
    away_total = (away_score.split(" ")[1][1:-1])  
    venue = test[4].find('a').string
    final_crowd = test[5].string
    match_report_url = match_report
    time = "No time provided"
    
    


    match_dict_= {
     "Day": day,
     "Date": date,
     "Month": month,
     "Year": this_year,
     "Round": this_round,         
     "Time": time,
     "Structured Data": struct_time,
     "Home Team": home_team,
     "Away Team": away_team,
     "Venue": venue,
     "Crowd": final_crowd,
     "Match Link": match_report_url,
     "Home Goals": home_goals,
     "Home Behinds": home_behinds,
     "Home Total": home_total,
     "Away Goals": away_goals,
     "Away Behinds": away_behinds,
     "Away Total": away_total}
    
    #print (match_dict)
    return (match_dict_)




### FUNCTION: MATCHDAY FOR LARGE TABLE TYPE

In [314]:
def match_function_A(match, this_round, this_year):


    try:

        namestocheck_good = ["Round", "Qualifying ", "Elimination", "Semi", "Preliminary", "Grand"]
        if any (name in this_round for name in namestocheck_good):
            if this_round == "Grand Final" or this_round == "Grand Final Replay":
                match_data = match[0]
            else:
                match_data = match

                 
  
            test = match_data.find_all('td')

            ### check if a score is return 
            test_for_score = (''.join(text for text in test[1].find_all(text=True) if text.parent.name != "a")).strip()
            if test_for_score:
                standard_dict = match_dictionary_A(test,this_round, this_year)
                scores_dict = scores_dictionary(test)
                dictionary = {**standard_dict, **scores_dict}
                return (dictionary)
            
            else: 

                standard_dict = match_dictionary_A(test,this_round, this_year)

                scores_dict = scores_dictionary_02(match[1])
                dictionary = {**standard_dict, **scores_dict}
                return (dictionary)

        
        else:
            print ("no name in scope")

    
    except Exception as e:
        print("this row does not have match data", e)


### FUNCTION: MATCHDAY FOR B- SMALL TABLE

In [315]:
def match_function_B(First_Columm, match,this_year):
    
    test = match.find_all('td')

    
    if First_Columm == 'Home team':

    
    
        if len(test) > 7:
            #print (len(test))
            match_report = test[-1].find('a')['href']
            #print (match_report)
            this_round = "Not Defined"
            test = test [:-1]
            #print (test)
            dictionary = match_dictionary_B(test, this_round, this_year, match_report)
            #print (dictionary)
            return (dictionary)
        else: 
            #print (len(test))
            #print ("normal")
            match_report = "No Match Report"
            this_round = "Not Defined"
            #print (test)
            dictionary = match_dictionary_B(test, this_round, this_year, match_report)
            #print (dictionary)
            return (dictionary)
 
    else: 

        if len(test) > 8:
            #match_report = test[10]
            match_report = test[-1].find('a')['href']
            this_round = test[0].string
            test = test [1:-2]
            dictionary = match_dictionary_B(test, this_round, this_year, match_report)
            #print (dictionary)
            return (dictionary)
        
        else: 
            match_report = "No Match Report"
            this_round = test[0].string
            test = test [1:]
            dictionary = match_dictionary_B(test, this_round, this_year, match_report)
            #print (dictionary)
            return (dictionary)



    


## Why are we not returning this dictioopnary property?
## maybe we could indent it?


### FUNCTION: CREATE DATA  FOR EACH ROUND (LIST TYPE A)

In [316]:
def round_function_A(round_number, year_data,current_year):    
    games= year_data[round_number].find_all('tr')
    ## Find the round header
    round_head = (''.join(text for text in games[1].find_all(text=True) if text.parent.name != "a")).strip()
    #print (round_head)
    #if isinstance(round_head,str) == False:
    ## Check is string is empty- empty strings are falsy- so we are asking if not false- double negative- not false = true
    if not round_head:
        round_head = games[1].find('a').string

    this_round = []
    
 
    if round_head == "Grand Final" or round_head == "Grand Final Replay":
        result= match_function_A(games[2:4],round_head, current_year)
        if result:
            this_round.append(result)
        return (this_round)


    else:  
        round_matchday = games[2:-1]
        for match in round_matchday:
            result= match_function_A(match, round_head, current_year)
            if result:
                this_round.append(result)
                
        return (this_round)

    #print ("this")

### FUNCTION: CREATE DATA  FOR EACH ROUND (LIST TYPE B)

In [317]:
def round_function_B(round_number, year_data,current_year):    
    games= year_data[round_number].find_all('tr')

    this_round = []
    
    
    First_Columm = games[0].find_all('td')[0].find('b').string
          
          
    round_matchday = games[1:]


    
    for match in round_matchday:
        result= match_function_B(First_Columm, match,current_year)
        #print (result)
        if result:
            #print (result)
            this_round.append(result)
    ## only working for finals games- check why they are different
    #print (this_round)
    return (this_round)



## TRY BOTH TABLE TYPES THROUGH EVERY YEAR (2001-2017)

### FUNCTION: LARGE TABLE- A

In [318]:
def table_typeA (year):    
    current_year = str(year)
    url = find_url(current_year)
    req = requests.get(url)
    soup = BeautifulSoup(req.text, 'html.parser')

    year_data = soup.find("div", {"id": "mw-content-text"}).find_all('table', {"style": "width: 100%; background-color: #f1f5fc; border: 2px solid #D0E5F5"})
    return (year_data)

### FUNCTION: SMALL TABLE- B

In [319]:
def table_typeB (year):    
    current_year = str(year)
    url = find_url(current_year)
    req = requests.get(url)
    soup = BeautifulSoup(req.text, 'html.parser')

    year_data= soup.find("div", {"id": "mw-content-text"}).find_all("table", {"class": "wikitable succession-box"})
    return (year_data)

### FUNCTION: CREATE DATA FOR EACH YEAR + TURN INTO 'MEGALIST'

In [369]:
mega_list = []
## 2007 NOT WORKING 2008 NOT WORKING
# 2002 is a mix
## 22004 only old
test_year = 2002

#for year in list(range(test_year,test_year + 1)):
for year in list(range(2000,2018,1)):

        ## Big Table
        year_data_A = table_typeA (year)
        for current_round in range(len(year_data_A)):
            #print (current_round)
            this_round = round_function_A(current_round, year_data_A, year)
            mega_list.extend(this_round)
            
            
        ## Small Table  
        year_data_B = table_typeB (year)
        for current_round in range(len(year_data_B)):
            this_round = round_function_B(current_round, year_data_B, year)
            mega_list.extend(this_round)
            
    
print (mega_list)


this row does not have match data list index out of range
this row does not have match data list index out of range
no name in scope
this row does not have match data time data '2011 11 March 7:45\xa0pm' does not match format '%Y %d %B (%I:%M %p)'
this row does not have match data 'NoneType' object has no attribute 'count'
this row does not have match data 'NoneType' object has no attribute 'count'
this row does not have match data 'NoneType' object has no attribute 'count'
this row does not have match data 'NoneType' object has no attribute 'count'
this row does not have match data 'NoneType' object has no attribute 'count'
this row does not have match data 'NoneType' object has no attribute 'count'
this row does not have match data list index out of range
this row does not have match data 'NoneType' object has no attribute 'count'
this row does not have match data 'NoneType' object has no attribute 'count'
this row does not have match data 'NoneType' object has no attribute 'count'
t

In [370]:
df = pd.DataFrame(mega_list)
#df.head(200)
df.shape

(3355, 18)

### BREAK

### TURN MEGALIST INTO PANDAS DATAFRAME + REVIEW

In [361]:
df = pd.DataFrame(mega_list)
#df.head(400)
df.shape

,Away Behinds,Away Goals,Away Team,Away Total,Crowd,Date,Day,Home Behinds,Home Goals,Home Team,Home Total,Match Link,Month,Round,Structured Data,Time,Venue,Year
0,6,13,Richmond,84,60654,24,Thursday,20,14,Carlton,104,https://web.archive.org/web/20110416022924/htt...,March,Round 1,2011-03-24 19:40:00,7:40 pm,MCG,2011
1,11,6,St Kilda,47,42869,25,Friday,12,6,Geelong,48,http://www.afl.com.au/news/newsarticle/tabid/2...,March,Round 1,2011-03-25 19:40:00,7:40 pm,MCG,2011
2,8,12,Port Adelaide,80,35773,26,Saturday,11,24,Collingwood,155,http://www.afl.com.au/news/newsarticle/tabid/2...,March,Round 1,2011-03-26 14:10:00,2:10 pm,Etihad Stadium,2011
3,13,12,Hawthorn,85,42536,26,Saturday,9,16,Adelaide,105,https://web.archive.org/web/20121007172516/htt...,March,Round 1,2011-03-26 19:10:00,7:10 pm,AAMI Stadium,2011
4,16,13,Fremantle,94,23373,26,Saturday,8,14,Brisbane Lions,92,https://web.archive.org/web/20121007172527/htt...,March,Round 1,2011-03-26 19:10:00,7:10 pm,The Gabba,2011
5,10,8,Western Bulldogs,58,42617,27,Sunday,17,16,Essendon,113,http://www.afl.com.au/news/newsarticle/tabid/2...,March,Round 1,2011-03-27 13:10:00,1:10 pm,Etihad Stadium,2011
6,18,11,Sydney,84,33951,27,Sunday,18,11,Melbourne,84,http://www.afl.com.au/news/newsarticle/tabid/2...,March,Round 1,2011-03-27 14:10:00,2:10 pm,MCG,2011
7,10,13,North Melbourne,88,35878,27,Sunday,14,13,West Coast,92,http://www.afl.com.au/news/newsarticle/tabid/2...,March,Round 1,2011-03-27 16:10:00,4:10 pm,Patersons Stadium,2011
8,11,14,Richmond,95,41465,1,Friday,17,13,St Kilda,95,https://web.archive.org/web/20110404113541/htt...,April,Round 2,2011-04-01 19:40:00,7:40 pm,MCG,2011
9,17,21,Collingwood,143,40578,2,Saturday,14,7,North Melbourne,56,http://www.afl.com.au/news/newsarticle/tabid/2...,April,Round 2,2011-04-02 14:10:00,2:10 pm,Etihad Stadium,2011


In [371]:
df.loc[df['Round'] == 'Grand Final']

,Away Behinds,Away Goals,Away Team,Away Total,Crowd,Date,Day,Home Behinds,Home Goals,Home Team,Home Total,Match Link,Month,Round,Structured Data,Time,Venue,Year
193,18,15,Brisbane Lions,108,91482,29,No Day Data Provided,10,12,Essendon,82,no match report,September,Grand Final,2001-09-29 14:30:00,2:30 pm,MCG,2001
563,12,12,Collingwood,84,79451,27,No Day Data Provided,14,20,Brisbane Lions,134,http://www.afl.com.au/news/newsarticle/tabid/2...,September,Grand Final,2003-09-27 14:40:00,2:40 pm,MCG,2003
924,13,10,Brisbane,73,"77,671",25,Saturday,11,17,Port Adelaide,113,No Match Report,September,Grand Final,2004-09-25 00:00:00,No time provided,MCG,2004
1112,13,12,West Coast,85,97431,30,Saturday,12,12,Sydney,84,http://www.afl.com.au/news/newsarticle/tabid/2...,September,Grand Final,2006-09-30 14:30:00,2:30 pm,MCG,2006
1664,7,18,Hawthorn,115,"100,012",27,Saturday,23,11,Geelong,89,http://www.afl.com.au/News/NEWSARTICLE/tabid/2...,September,Grand Final,2008-09-27 00:00:00,No time provided,MCG,2008
1849,8,12,Geelong,80,99251,26,No Day Data Provided,14,9,St Kilda,68,http://www.afl.com.au/news/newsarticle/tabid/2...,September,Grand Final,2009-09-26 14:30:00,2:30 pm,MCG,2009
2034,8,10,St Kilda,68,100016,25,No Day Data Provided,14,9,Collingwood,68,http://www.afl.com.au/news/newsarticle/tabid/2...,September,Grand Final,2010-09-25 14:30:00,2:30 pm,MCG,2010
2231,11,18,Geelong,119,99537,1,Saturday,9,12,Collingwood,81,http://www.afl.com.au/news/newsarticle/tabid/2...,October,Grand Final,2011-10-01 14:30:00,2:30 pm,MCG,2011
2438,7,14,Sydney,91,99683,29,Saturday,15,11,Hawthorn,81,http://www.afl.com.au/match-centre/2012/27/haw...,September,Grand Final,2012-09-29 14:30:00,2:30 pm,MCG,2012
2645,14,8,Fremantle,62,100007,28,Saturday,11,11,Hawthorn,77,http://www.afl.com.au/match-centre/2013/27/haw...,September,Grand Final,2013-09-28 14:30:00,2:30 pm,MCG,2013


In [375]:
df.loc[896]

Away Behinds                         9
Away Goals                          23
Away Team                Port Adelaide
Away Total                         147
Crowd                           13,271
Date                                14
Day                           Saturday
Home Behinds                         7
Home Goals                           9
Home Team             Western Bulldogs
Home Total                          61
Match Link             No Match Report
Month                           August
Round                      Not Defined
Structured Data    2004-08-14 00:00:00
Time                  No time provided
Venue                     Marrara Oval
Year                              2004
Name: 896, dtype: object

### IDENTIFY UNIQUE VENUE TYPES

# We will want to rerun this and see if we have any new ones to put into consistent dict- then we will want to add in any stations we may need- check they all work to 2001 and run the script to populate

In [372]:
df.Venue.unique()

array(['MCG', 'Colonial Stadium', 'Football Park', 'Subiaco Oval',
       'Optus Oval', 'Shell Stadium', 'SCG', 'WACA', 'Gabba', 'The Gabba',
       'York Park', 'Manuka Oval', 'Skilled Stadium', 'Stadium Australia',
       'AAMI Stadium', 'Telstra Stadium', 'Telstra Dome', 'Marrara Oval',
       'Aurora Stadium', 'Carrara Stadium', 'ANZ Stadium',
       'Etihad Stadium', 'TIO Stadium', 'Gold Coast Stadium', 'Subiaco',
       'Patersons Stadium', 'Metricon Stadium', "Cazaly's Stadium",
       'Adelaide Oval', 'Blundstone Arena', 'Blacktown ISP',
       'Simonds Stadium', 'Škoda Stadium', 'StarTrack Oval Canberra',
       'Westpac Stadium', 'Spotless Stadium', 'StarTrack Oval',
       'TIO Traeger Park', 'Domain Stadium', 'UNSW Canberra Oval',
       'University of Tasmania Stadium', 'Jiangwan Stadium'], dtype=object)

### CREATE CONSISTENT VENUE TYPES (NO DOUBLE UPS)

In [378]:
def rationalise_venue(row):
    di_cities= {"MCG": "MCG",
     "The Gabba": "Gabba",
     "Gabba": "Gabba",
     "Colonial Stadium": "Etihad Stadium",
     "Etihad Stadium": "Etihad Stadium",
     "Telstra Dome": "Etihad Stadium",
     "Optus Oval": "Optus Oval",
     "Adelaide Oval": "Adelaide Oval",
     "Manuka Oval": "Manuka Oval",
     "UNSW Canberra Oval": "Manuka Oval",
     "StarTrack Oval": "Manuka Oval",
     "StarTrack Oval Canberra": "Manuka Oval",
     "Domain Stadium": "Domain Stadium",
     "Subiaco Oval": "Domain Stadium",
     "Subiaco": "Domain Stadium",
     "Patersons Stadium": "Domain Stadium",
     "Metricon Stadium": "Metricon Stadium",
     "Carrara Stadium": "Metricon Stadium",
     "Gold Coast Stadium": "Metricon Stadium",
     "WACA": "WACA",
     "Blundstone Arena": "Blundstone Arena",
     "SCG": "SCG",
     "Spotless Stadium": "Spotless Stadium",
     "Blacktown ISP": "Spotless Stadium",
     "Škoda Stadium": "Spotless Stadium",
     "ANZ Stadium": "ANZ Stadium",
     "Telstra Stadium": "ANZ Stadium",
     "Stadium Australia": "ANZ Stadium",
     "Simonds Stadium": "Simonds Stadium",
     "Skilled Stadium": "Simonds Stadium",
     "Shell Stadium": "Simonds Stadium",
     "York Park": "University of Tasmania Stadium",
     "Aurora Stadium": "University of Tasmania Stadium",
     "University of Tasmania Stadium": "University of Tasmania Stadium",
     "Cazaly's Stadium": "Cazaly's Stadium",
     "TIO Traeger Park": "TIO Traeger Park",
     "TIO Stadium": "TIO Stadium",    
     "Marrara Oval": "TIO Stadium",    
     "Westpac Stadium": "Westpac Stadium",
     "Football Park": "AAMI Stadium",
     "AAMI Stadium": "AAMI Stadium",
     "Jiangwan Stadium": "Jiangwan Stadium",
     "Westpac Stadium": "Westpac Stadium"}
    
    return di_cities[row.Venue]

In [379]:
df["Venue"] = df.apply(rationalise_venue, axis=1)
df.Venue.unique()

array(['MCG', 'Etihad Stadium', 'AAMI Stadium', 'Domain Stadium',
       'Optus Oval', 'Simonds Stadium', 'SCG', 'WACA', 'Gabba',
       'University of Tasmania Stadium', 'Manuka Oval', 'ANZ Stadium',
       'TIO Stadium', 'Metricon Stadium', "Cazaly's Stadium",
       'Adelaide Oval', 'Blundstone Arena', 'Spotless Stadium',
       'Westpac Stadium', 'TIO Traeger Park', 'Jiangwan Stadium'], dtype=object)

### CREATE CITIES LIST BASED ON VENUES

In [384]:
def get_city(row):
    di_cities= {"MCG": "Melbourne",
     "Gabba": "Brisbane",
     "Etihad Stadium": "Melbourne",
     "Optus Oval": "Adelaide",
     "Adelaide Oval": "Adelaide",
     "Manuka Oval": "Canberra",
     "WACA": "Perth",
     "Domain Stadium": "Perth",
     "Metricon Stadium": "Gold Coast",
     "Blundstone Arena": "Hobart",
     "SCG": "Sydney",
     "Spotless Stadium": "Western Sydney",
     "ANZ Stadium": "Western Sydney",
     "Simonds Stadium": "Geelong",
     "University of Tasmania Stadium": "Launceston",
     "Cazaly's Stadium": "Cairns",
     "TIO Traeger Park": "Alice Springs",
     "TIO Stadium": "Darwin",    
     "Westpac Stadium": "Wellington",
     "AAMI Stadium": "Adelaide",
     "Jiangwan Stadium": "Shanghai"}
                
    return di_cities[row.Venue]

In [385]:
df["City"] = df.apply(get_city, axis=1)

In [404]:
df.head()

,Away Behinds,Away Goals,Away Team,Away Total,Crowd,Date,Day,Home Behinds,Home Goals,Home Team,Home Total,Match Link,Month,Round,Structured Data,Time,Venue,Year,City
0,14,13,Melbourne,92,"41,648",8,Wednesday,10,14,Richmond,94,No Match Report,March,Not Defined,2000-03-08,No time provided,MCG,2000,Melbourne
1,14,8,Port Adelaide,62,"43,012",9,Thursday,12,24,Essendon,156,No Match Report,March,Not Defined,2000-03-09,No time provided,Etihad Stadium,2000,Melbourne
2,10,24,West Coast,154,"20,580",10,Friday,15,16,Kangaroos,111,No Match Report,March,Not Defined,2000-03-10,No time provided,MCG,2000,Melbourne
3,17,19,Western Bulldogs,131,"37,222",11,Saturday,18,15,Adelaide,108,No Match Report,March,Not Defined,2000-03-11,No time provided,AAMI Stadium,2000,Adelaide
4,15,19,Geelong,129,"19,777",11,Saturday,11,16,Fremantle,107,No Match Report,March,Not Defined,2000-03-11,No time provided,Domain Stadium,2000,Perth


### CREATE WEATHER STATION ID'S LIST BASED ON CITIES

In [435]:
def get_station_ID(row):
    di_stations= {
    "Melbourne": "1487228352&p_stn_num=086232",
     "Brisbane": "334811501&p_stn_num=040913",
     "Adelaide": "106666407&p_stn_num=023090",
     "Canberra": "986936890&p_stn_num=070246",
     "Perth": "17056912&p_stn_num=009225",         
     "Gold Coast": "323084007&p_stn_num=040190",
     "Hobart": "1768364967&p_stn_num=094030",
     "Sydney": "872874356&p_stn_num=066062",
     "Western Sydney": "872504448&p_stn_num=066048",
     "Geelong": "1515020218&p_stn_num=087034",
     "Launceston": "1658858624&p_stn_num=091072",
     "Cairns": "194300698&p_stn_num=031166",
     "Alice Springs": "48646407&p_stn_num=015590",
     "Darwin": "40154901&p_stn_num=014163",    
     "Wellington": "N/A",
     "Shanghai": "N/A"}
                
    return di_stations[row.City]


### copy city links from below

In [436]:
df["station_ID"] = df.apply(get_station_ID, axis=1)

### CONVERT MONTH

In [438]:
def month(row):
    month_dict = {
    "March" : "3",
    "April" : "4",
    "May" : "5",
    "June" : "6",
    "July" : "7",
    "August" : "8",
    "September" : "9",
    "October" : "10",
    }
    
    return month_dict[row.Month]

In [439]:
df["Month_No"] = df.apply(month, axis=1)

In [440]:
df.head()

,Away Behinds,Away Goals,Away Team,Away Total,Crowd,Date,Day,Home Behinds,Home Goals,Home Team,...,Month,Round,Structured Data,Time,Venue,Year,City,station_ID,URL,Month_No
0,14,13,Melbourne,92,"41,648",8,Wednesday,10,14,Richmond,...,March,Not Defined,2000-03-08,No time provided,MCG,2000,Melbourne,1487228352&p_stn_num=086232,http://www.bom.gov.au/jsp/ncc/cdio/weatherData...,3
1,14,8,Port Adelaide,62,"43,012",9,Thursday,12,24,Essendon,...,March,Not Defined,2000-03-09,No time provided,Etihad Stadium,2000,Melbourne,1487228352&p_stn_num=086232,http://www.bom.gov.au/jsp/ncc/cdio/weatherData...,3
2,10,24,West Coast,154,"20,580",10,Friday,15,16,Kangaroos,...,March,Not Defined,2000-03-10,No time provided,MCG,2000,Melbourne,1487228352&p_stn_num=086232,http://www.bom.gov.au/jsp/ncc/cdio/weatherData...,3
3,17,19,Western Bulldogs,131,"37,222",11,Saturday,18,15,Adelaide,...,March,Not Defined,2000-03-11,No time provided,AAMI Stadium,2000,Adelaide,106666407&p_stn_num=023090,http://www.bom.gov.au/jsp/ncc/cdio/weatherData...,3
4,15,19,Geelong,129,"19,777",11,Saturday,11,16,Fremantle,...,March,Not Defined,2000-03-11,No time provided,Domain Stadium,2000,Perth,17056912&p_stn_num=009225,http://www.bom.gov.au/jsp/ncc/cdio/weatherData...,3


In [443]:
def find_url(row):
    Base_Link_01= "http://www.bom.gov.au/jsp/ncc/cdio/weatherData/av?p_nccObsCode=136&p_display_type=dailyDataFile&p_startYear="
    Base_Link_02= "&p_c=-"
    link = Base_Link_01 + str(row.Year) + Base_Link_02 + str(row.station_ID)
    return link
    #print (link)

In [444]:
df["URL"] = df[(df.City != 'Wellington') & (df.City != 'Shanghai')].apply(find_url, axis=1)

In [445]:
df.head()

,Away Behinds,Away Goals,Away Team,Away Total,Crowd,Date,Day,Home Behinds,Home Goals,Home Team,...,Month,Round,Structured Data,Time,Venue,Year,City,station_ID,URL,Month_No
0,14,13,Melbourne,92,"41,648",8,Wednesday,10,14,Richmond,...,March,Not Defined,2000-03-08,No time provided,MCG,2000,Melbourne,1487228352&p_stn_num=086232,http://www.bom.gov.au/jsp/ncc/cdio/weatherData...,3
1,14,8,Port Adelaide,62,"43,012",9,Thursday,12,24,Essendon,...,March,Not Defined,2000-03-09,No time provided,Etihad Stadium,2000,Melbourne,1487228352&p_stn_num=086232,http://www.bom.gov.au/jsp/ncc/cdio/weatherData...,3
2,10,24,West Coast,154,"20,580",10,Friday,15,16,Kangaroos,...,March,Not Defined,2000-03-10,No time provided,MCG,2000,Melbourne,1487228352&p_stn_num=086232,http://www.bom.gov.au/jsp/ncc/cdio/weatherData...,3
3,17,19,Western Bulldogs,131,"37,222",11,Saturday,18,15,Adelaide,...,March,Not Defined,2000-03-11,No time provided,AAMI Stadium,2000,Adelaide,106666407&p_stn_num=023090,http://www.bom.gov.au/jsp/ncc/cdio/weatherData...,3
4,15,19,Geelong,129,"19,777",11,Saturday,11,16,Fremantle,...,March,Not Defined,2000-03-11,No time provided,Domain Stadium,2000,Perth,17056912&p_stn_num=009225,http://www.bom.gov.au/jsp/ncc/cdio/weatherData...,3


In [455]:
# RETURN THE WEATHER 
def find_rain(row):
    req = requests.get(row.URL)
    soup = BeautifulSoup(req.text, 'html.parser')
    
    # FIND ALL TR TAGS IN TBODY BESIDES ONES WITH A TR CLASS NOT EQUAL TO 'GRAPHCELL'
    Relevent_Data = soup.find('tbody').find_all('tr', class_=lambda x: x != 'graphcell')

    # CALL THE RELEVENT DAY TR LIST WITH INDEX NUMBER (EG. 3rd = index 2)
    Day_Rain = Relevent_Data[(int(row.Date) - 1)]
    
    # FIND ALL THE 'TD' OR MONTH TAGS FOR A GIVEN DAY OF THE MONTH
    All_Months_Rain = Day_Rain.find_all('td')
    
    # ASSIGN THE MONTH OF A GIVEN ROW
    Month_Value = (int(row.Month_No) - 1)

    # CALL THE RELEVENT MONTH TD LIST WITH INDEX NUMBER (note. we have already created index with necessary index)
    Rain = (All_Months_Rain[Month_Value].string)
    print (Rain)
    return Rain
    #print (Rain)



In [456]:
df["Rain"] = df[(df.City != 'Wellington') & (df.City != 'Shanghai')].apply(find_rain, axis=1)

0
0
0
0
0
0
0
0
0
0
0.4
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2.0
0
0
0
0
0
4.2
0.8
2.8


KeyboardInterrupt: 

In [ ]:
df.head()

### SELECT ROW BASED ON COLUMN VALUE (EG.VENUE IS METRICON)

df.loc[df['Venue'] == 'Metricon Stadium']

### EXPORT DATA TO CSV

In [143]:
df.to_csv("test.csv")